In [151]:
import pyautogui
import time
import webbrowser
import pandas as pd
import bs4
import pyperclip
import numpy as np
import os
import re
import math
from datetime import datetime, timedelta


from urllib.parse import urlparse
#%% Windows simulation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.common.exceptions import NoSuchElementException



In [3]:
#读取txt文件 文件内容是ICO表格的html
html_path = "../data/ICOpage"
files = [f for f in os.listdir(html_path) if not f.startswith('.')]
files.sort()
print(f'page number: {len(files)}')


page number: 8


In [1]:
def act_table(source_codes):
     soup = bs4.BeautifulSoup(source_codes, 'html.parser')
     elems = soup.find_all('tr')
     
     # 寻找表头
     head_elem = elems[0]  # 假设表头在第一个<tr>元素中
     head_subelems = head_elem.select('th')  # 如果表头是用<th>标签表示的，请使用'th'
     head = [re.sub("\n","",i.text) for i in head_subelems]
     head = [head[0]]+['Abbr','Link']+head[1:]

     rows = []

     for elem in elems[1:]:  # 跳过第一行，因为它是表头
          
          subelems = elem.select('td')
          project_name = elem.select_one('.sc-151230bb-0.bgIAsX').text.strip()

          # Find the abbreviation
          abbreviation = elem.select_one('.sc-151230bb-0.gmIrUD').text.strip()
          
          class_a = elem.select_one('a.sc-77da7dd7-1.dTUxdJ')
          try: 
               if class_a:
                    link = 'https://cryptorank.io'+class_a.get('href')
                    
               else:
                    link = None
          except TypeError as error:
               print('can not get the href')
               link = None
               
          

          #print(f"Project Name: {project_name}, Abbreviation: {abbreviation}")
          row = [i.text for i in subelems[1:]]
          row = [project_name,abbreviation,link]+row
          rows.append(row)

     df = pd.DataFrame(rows, columns=head)
     
    
     return df
     

In [188]:
ico_table = pd.DataFrame()
for file in files:
     path = f"{html_path}/{file}"
     with open(path, 'r', encoding='utf-8') as file:
          html_content = file.read()
     tb = act_table(html_content)
     df = pd.DataFrame(tb)
     
     ico_table = pd.concat([ico_table, df], axis=0, ignore_index=True)
     
ico_table

,Project,Abbr,Link,Sale Price,ROI,Total Raise,Launchpad,Ended,When,Funds and Investors,Type,Raise,Initial Cap,Market Cap,Blockchain,Category,Listing Date,ATH ROI
0,Artyfact,ARTY,https://cryptorank.io/ico/artyfact,$ 0.300,N/A,$ 2.45M,TruePNL,30 Oct,11 Oct 2022,,"ICO, IDO, Seed, Private","$ 1,370,000",$ 216.00K,N/A,BNB,GameFi,TBA,N/A
1,SupraOracles,SUPRA,https://cryptorank.io/ico/supra-oracles,$ 0.048,N/A,$ 2.40M,,26 Oct,25 Oct,Coinbase Ventures+21,"ICO, Strategic, Private, Seed, Pre-Seed, Priva...","$ 2,400,000",N/A,N/A,,Blockchain Infrastructure,TBA,N/A
2,Islamic Coin,ISLM,https://cryptorank.io/ico/islamic-coin,$ 0.300,0.60x,$ 2.40M,Republic,9 Oct,12 Sep,,ICO,"$ 2,399,474",N/A,N/A,,Chain,10 Oct,1.20x
3,Archway,ARCH,https://cryptorank.io/ico/archway,$ 0.200,0.38x,$ 6.03M,Coinlist+1,27 Sep,15 Jun,Blockchain Capital+11,"ICO, IEO","$ 6,030,000",N/A,$ 3.81M,,Chain,28 Sep,0.46x
4,ScapesMania,$MANIA,https://cryptorank.io/ico/scapesmania,$ 0.009,N/A,$ 5.87M,,10 Sep,28 Aug,,"Private/Pre-sale, ICO","$ 720,000",N/A,N/A,BNB+1,GameFi,TBA,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,Waves,WAVES,https://cryptorank.io/ico/waves,$ 0.188,12.01x,$ 16.00M,,TBA,TBA,,ICO,"$ 16,000,000",$ 18.80M,$ 225.86M,Ethereum,Chain,2 Jun 2016,333.28x
790,WeTrust,TRST,https://cryptorank.io/ico/trust,$ 0.06,0.01x,N/A,,TBA,TBA,,ICO,N/A,N/A,$ 70.70K,Ethereum,,20 Apr 2017,23.34x
791,ZenGold,ZENGOLD,https://cryptorank.io/ico/zengold,$ 3.86,N/A,N/A,,TBA,TBA,,ICO,N/A,N/A,N/A,BitShares,Social,TBA,N/A
792,Zipper,ZIP,https://cryptorank.io/ico/zip,$ 0.03,0.00x,N/A,,TBA,TBA,,ICO,N/A,N/A,$ 123.21K,Ethereum,Blockchain Infrastructure,29 May 2018,0.28x


In [189]:
def price_clean(price):
     price = re.sub('\$','',price)
     price = re.sub('\s','',price)
     price = re.sub('\,','',price)
     if price == '0.00...004':
          price = '0.000004'
     tail = price[-1]
     if tail =='A':
          price = None
     elif tail == 'B':
          price = float(price[:-1])*1000000000
     elif tail == 'M':
          price = float(price[:-1])*1000000
     elif tail == 'K':
          price = float(price[:-1])*1000
     else:
          price = float(price)
 
     return price
def roi_clean(roi):
     roi = re.sub('x','',roi)
     if roi=='N\A':
          roi = None
     
     return roi

def date_clean(date_str):
     if date_str == 'TBA':
        return None
     try:
          # Assuming date_str is either "DD Mon" or "DD Mon YYYY"
          return pd.to_datetime(date_str, format='%d %b', errors='raise').replace(year=2023)
     except ValueError:
          # If parsing fails, try the other format
          return pd.to_datetime(date_str, format='%d %b %Y', errors='raise')
def ICO_number(lst):
     #print(list(enumerate(lst)))
     for id, value in enumerate(lst):
          #print(f'index:{id},value:{value}')
          if value=='ICO':
               return id+1 
     return -1


In [190]:
def df_clean(df):
     
     df['Sale Price'] = df['Sale Price'].apply(price_clean)
     df['Total Raise'] = df['Total Raise'].apply(price_clean)
     df['Raise'] = df['Raise'].apply(price_clean)
     df['Initial Cap'] = df['Initial Cap'].apply(price_clean)
     df['Market Cap'] = df['Market Cap'].apply(price_clean)
     df['ROI'] = df['ROI'].apply(roi_clean)
     df['ATH ROI'] = df['ATH ROI'].apply(roi_clean)
     df['When'] = df['When'].apply(date_clean)
     df['Ended'] = df['Ended'].apply(date_clean)
     df['ICO Days'] = df['Ended']-df['When']+ timedelta(days=1)
     df['Listing Date'] = df['Listing Date'].apply(date_clean)
     df['Type'] = df['Type'].apply(lambda x:x.split(', '))
     df['Rounds'] =df['Type'].apply(len)
     df['ICO Order'] = df['Type'].apply(ICO_number)
     return df

In [191]:
## 数据清洗+保存
ico_table_cleaned = df_clean(ico_table)
ico_table_cleaned.to_csv("../data/ico_info.csv")
ico_table_cleaned

,Project,Abbr,Link,Sale Price,ROI,Total Raise,Launchpad,Ended,When,Funds and Investors,...,Raise,Initial Cap,Market Cap,Blockchain,Category,Listing Date,ATH ROI,ICO Days,Rounds,ICO Order
0,Artyfact,ARTY,https://cryptorank.io/ico/artyfact,0.300,N/A,2450000.0,TruePNL,2023-10-30,2022-10-11,,...,1370000.0,216000.0,NaN,BNB,GameFi,NaT,N/A,385 days,4,1
1,SupraOracles,SUPRA,https://cryptorank.io/ico/supra-oracles,0.048,N/A,2400000.0,,2023-10-26,2023-10-25,Coinbase Ventures+21,...,2400000.0,NaN,NaN,,Blockchain Infrastructure,NaT,N/A,2 days,6,1
2,Islamic Coin,ISLM,https://cryptorank.io/ico/islamic-coin,0.300,0.60,2400000.0,Republic,2023-10-09,2023-09-12,,...,2399474.0,NaN,NaN,,Chain,2023-10-10,1.20,28 days,1,1
3,Archway,ARCH,https://cryptorank.io/ico/archway,0.200,0.38,6030000.0,Coinlist+1,2023-09-27,2023-06-15,Blockchain Capital+11,...,6030000.0,NaN,3810000.0,,Chain,2023-09-28,0.46,105 days,2,1
4,ScapesMania,$MANIA,https://cryptorank.io/ico/scapesmania,0.009,N/A,5870000.0,,2023-09-10,2023-08-28,,...,720000.0,NaN,NaN,BNB+1,GameFi,NaT,N/A,14 days,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,Waves,WAVES,https://cryptorank.io/ico/waves,0.188,12.01,16000000.0,,NaT,NaT,,...,16000000.0,18800000.0,225860000.0,Ethereum,Chain,2016-06-02,333.28,NaT,1,1
790,WeTrust,TRST,https://cryptorank.io/ico/trust,0.060,0.01,NaN,,NaT,NaT,,...,NaN,NaN,70700.0,Ethereum,,2017-04-20,23.34,NaT,1,1
791,ZenGold,ZENGOLD,https://cryptorank.io/ico/zengold,3.860,N/A,NaN,,NaT,NaT,,...,NaN,NaN,NaN,BitShares,Social,NaT,N/A,NaT,1,1
792,Zipper,ZIP,https://cryptorank.io/ico/zip,0.030,0.00,NaN,,NaT,NaT,,...,NaN,NaN,123210.0,Ethereum,Blockchain Infrastructure,2018-05-29,0.28,NaT,1,1


In [255]:
def match_links(link_list):
    links = {}
    #link_list = ['https://supraoracles.com/', 'https://twitter.com/SupraOracles?s=20', 'https://t.me/supraoracles', 'https://discord.gg/supraoracles', 'https://medium.com/@SupraOracles']
    for link in link_list:
        if "bitcointalk.org" in link:
            links['Website'] = link
        elif "twitter.com" in link:
            links['Twitter'] = link
        elif "t.me" in link:
            links['Telegram'] = link
        elif "discord.gg" in link:
            links['Discord'] = link
        elif "medium.com" in link:
            links['Medium'] = link 
    return links

In [242]:
link_table = ico_table[['Project','Link']]
link_table['Social'] = [{} for _ in range(len(link_table))]


In [310]:
#subpage的提取工作

# Iterate through rows and update 'Social' based on 'Link'
for index, row in link_table.iloc[776:].iterrows():
     
     lnk = row['Link']
     if index %30 == 0:
          print(f'Index:{index}. Now working on: {lnk}')
     links = {}
     driver = webdriver.Chrome()
     wait = WebDriverWait(driver, 10)
     actions = ActionChains(driver)
     time.sleep(1)
     driver.get(lnk)
     driver.implicitly_wait(2)
     # 使用XPath定位元素
     socials_element = driver.find_element(By.XPATH,'//*[@id="root-container"]/div[2]/div[1]/div[3]/div[2]')
     
     # 获取p标签的文本内容
     p_text = socials_element.find_element(By.TAG_NAME,'p').text
     
     # 判断是否为'Socials'
     if p_text == 'Socials':
          try:
               button = socials_element.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div[1]/div[3]/div[2]/div/button')
          except NoSuchElementException:
               button = None

          if button:
               button.click()
               time.sleep(1)
               hidden_links = driver.find_element(By.CSS_SELECTOR, 'body > div.app-modal > div > div > div.sc-a682bf5d-5.YHcgw > div > div')
               items = hidden_links.find_elements(By.TAG_NAME, 'a')

               links = {}
               for item in items:
                    # 获取<p>标签的文本内容
                    paragraph_text = item.text
                    # 获取链接
                    link = item.get_attribute('href')
                    links[paragraph_text] = link
          else:
               print(f'review:{index}. Whoes link is {lnk}')
               social_links = socials_element.find_elements(By.TAG_NAME, 'a')
               social_links_url = []
               # 提取链接并打印
               for item in social_links:
                    href = item.get_attribute('href')
                    social_links_url.append(href)
               #links = match_links(social_links_url)
               links = social_links_url
                    
               
     else:
          print("This is not a 'Socials' element")
     link_table.at[index, 'Social'] = links
     driver.quit()
     time.sleep(3)
#link_table.to_csv("../data/ico_sociallinks.csv")

     



review:777. Whoes link is https://cryptorank.io/ico/starcointv
review:778. Whoes link is https://cryptorank.io/ico/storjcoin-x
Index:780. Now working on: https://cryptorank.io/ico/taas
review:781. Whoes link is https://cryptorank.io/ico/tenx
review:782. Whoes link is https://cryptorank.io/ico/ternio
review:784. Whoes link is https://cryptorank.io/ico/todaycoin
review:787. Whoes link is https://cryptorank.io/ico/universal-reward-protocol
review:788. Whoes link is https://cryptorank.io/ico/vulcan-forged-pyr
review:791. Whoes link is https://cryptorank.io/ico/zengold
review:792. Whoes link is https://cryptorank.io/ico/zip


In [323]:
link_table = pd.merge(ico_table_cleaned,link_table,on=['Project','Link'])
link_table
#link_table.to_csv("../data/ico_sociallinks.csv")#\count = 0
count =0
investor_table = ico_table_cleaned[['Project','Link','Funds and Investors']]

In [362]:

for index , row in investor_table.iloc[100:].iterrows():
     investor = row['Funds and Investors']
     lnk = row['Link']
     if len(investor) != 0:
          count +=1
          print(f'Now working on {lnk}, investor:{investor}')
          last_letter = investor[-1]
          if last_letter >= '0' and last_letter <= '9':
               investor_list = []
               driver = webdriver.Chrome()
               wait = WebDriverWait(driver, 10)
               actions = ActionChains(driver)
               time.sleep(1)
               driver.get(lnk)
               driver.implicitly_wait(2)
               time.sleep(5)
               # 使用XPath定位元素
               
               x_path = '//*[@id="root-container"]/div[2]/div[1]/div[3]/div[3]'
               investor_element = driver.find_element(By.XPATH,x_path)
               
               # 获取p标签的文本内容
               p_text = investor_element.find_element(By.TAG_NAME,'p').text
               
               
               if p_text == 'Socials':
                    x_path = '//*[@id="root-container"]/div[2]/div[1]/div[3]/div[4]'
                    investor_element = driver.find_element(By.XPATH,x_path)
               
                    # 获取p标签的文本内容
                    p_text = investor_element.find_element(By.TAG_NAME,'p').text
                    
               
               # 判断是否为'Funds and Backers'
               if p_text == 'Funds and Backers':
                    try:
                         button = investor_element.find_element(By.XPATH, x_path+'/div/button')
                    except NoSuchElementException:
                         button = None

                    if button:
                         button.click()
                         time.sleep(1)
                         hidden_links = driver.find_element(By.CSS_SELECTOR, 'body > div.app-modal > div > div > div.sc-a682bf5d-5.bAIVsn > div.sc-fe00af54-3.fleCiY')
                         items = hidden_links.find_elements(By.TAG_NAME, 'a')

                         
                         for item in items:
                              # 获取<p>标签的文本内容
                              paragraph_text = item.text
                              # 获取链接
                              investor_list.append(paragraph_text)
                         
                    else:
                         
                         #investor_element = investor_element.find_elements(By.TAG_NAME, 'a')
                         a_element = investor_element.find_elements(By.TAG_NAME, 'a')
                         

                         for item in a_element:
                              try:
                                   p_tag = item.find_element(By.TAG_NAME, 'p')

                                   investor_list.append(p_tag.text)
                                   

                              except NoSuchElementException:
                                   # If 'p' tag is not found, print the 'alt' attribute value
                                   alt_value = item.find_element(By.TAG_NAME, 'img').get_attribute('alt')
                                   investor_list.append(alt_value)
                                   
                         
                         
                              
                         
               else:
                    print("This is not a 'Funds' element")
               investor_table.at[index,'Funds and Investors'] = investor_list
               print(f'investor_list:{investor_list}')
               driver.quit()
               time.sleep(3)
          else:
               investor_table.at[index,'Funds and Investors'] = [investor]
print(f'count number {count}')
investor_table

Now working on https://cryptorank.io/ico/dexe-network, investor:Bitrise Capital+6
investor_list:['Waterdrip Capital', 'Chain Capital', 'Bitrise Capital', 'Quest Capital', 'Consensus Lab', 'BN Capital', 'Nova Club']
Now working on https://cryptorank.io/ico/flow, investor:Andreessen Horowitz (a16z)+11
investor_list:['Coinbase Ventures', 'Andreessen Horowitz (A16z)', 'Multicoin Capital', 'Animoca Brands', 'Digital Currency Group (DCG)', 'Fenbushi Capital', 'Dapper Labs', 'Republic', 'LedgerPrime', 'Greenfield Capital', 'Liberty City Ventures', 'Coatue Management']
Now working on https://cryptorank.io/ico/powertrade, investor:Pantera Capital+10
investor_list:['Pantera Capital\nLead', 'CMS Holdings', 'Framework Ventures', 'Signum Capital', 'LedgerPrime', 'QCP Capital', '3Commas', 'Kain Warwick', 'Taureon Capital', 'BoxOne Ventures', 'Loi Luu']
Now working on https://cryptorank.io/ico/dhedge, investor:DeFiance Capital+11
investor_list:['Framework Ventures', 'Mechanism Capital', 'DeFiance Cap

,Project,Link,Funds and Investors
0,Artyfact,https://cryptorank.io/ico/artyfact,
1,SupraOracles,https://cryptorank.io/ico/supra-oracles,"[Coinbase Ventures, HashKey Capital, Animoca B..."
2,Islamic Coin,https://cryptorank.io/ico/islamic-coin,
3,Archway,https://cryptorank.io/ico/archway,"[Blockchain Capital, Hashed Fund\nLead, CoinFu..."
4,ScapesMania,https://cryptorank.io/ico/scapesmania,
...,...,...,...
789,Waves,https://cryptorank.io/ico/waves,
790,WeTrust,https://cryptorank.io/ico/trust,
791,ZenGold,https://cryptorank.io/ico/zengold,
792,Zipper,https://cryptorank.io/ico/zip,


In [371]:
link_table = link_table.rename(columns={'Funds and Investors': 'Funds'})
investor_table  = pd.merge(link_table,investor_table,on=['Project','Link'])
investor_table 


,Project,Abbr,Link,Sale Price,ROI,Total Raise,Launchpad,Ended,When,Funds,...,Market Cap,Blockchain,Category,Listing Date,ATH ROI,ICO Days,Rounds,ICO Order,Social,Funds and Investors
0,Artyfact,ARTY,https://cryptorank.io/ico/artyfact,0.300,N/A,2450000.0,TruePNL,2023-10-30,2022-10-11,,...,NaN,BNB,GameFi,NaT,N/A,385 days,4,1,"{'Website': 'https://artyfact.game/', 'X (Ex-T...",
1,SupraOracles,SUPRA,https://cryptorank.io/ico/supra-oracles,0.048,N/A,2400000.0,,2023-10-26,2023-10-25,Coinbase Ventures+21,...,NaN,,Blockchain Infrastructure,NaT,N/A,2 days,6,1,"{'Website': 'https://supraoracles.com/', 'X (E...","[Coinbase Ventures, HashKey Capital, Animoca B..."
2,Islamic Coin,ISLM,https://cryptorank.io/ico/islamic-coin,0.300,0.60,2400000.0,Republic,2023-10-09,2023-09-12,,...,NaN,,Chain,2023-10-10,1.20,28 days,1,1,"{'Website': 'https://islamiccoin.net/', 'X (Ex...",
3,Archway,ARCH,https://cryptorank.io/ico/archway,0.200,0.38,6030000.0,Coinlist+1,2023-09-27,2023-06-15,Blockchain Capital+11,...,3810000.0,,Chain,2023-09-28,0.46,105 days,2,1,"[https://archway.io/, https://twitter.com/arch...","[Blockchain Capital, Hashed Fund\nLead, CoinFu..."
4,ScapesMania,$MANIA,https://cryptorank.io/ico/scapesmania,0.009,N/A,5870000.0,,2023-09-10,2023-08-28,,...,NaN,BNB+1,GameFi,NaT,N/A,14 days,2,2,{'Website': 'https://scapesmania.io/?utm_sourc...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,Waves,WAVES,https://cryptorank.io/ico/waves,0.188,12.01,16000000.0,,NaT,NaT,,...,225860000.0,Ethereum,Chain,2016-06-02,333.28,NaT,1,1,"{'Website': 'https://waves.tech/', 'X (Ex-Twit...",
790,WeTrust,TRST,https://cryptorank.io/ico/trust,0.060,0.01,NaN,,NaT,NaT,,...,70700.0,Ethereum,,2017-04-20,23.34,NaT,1,1,"{'Website': 'https://www.wetrust.io/', 'X (Ex-...",
791,ZenGold,ZENGOLD,https://cryptorank.io/ico/zengold,3.860,N/A,NaN,,NaT,NaT,,...,NaN,BitShares,Social,NaT,N/A,NaT,1,1,"[http://www.zengold.org/, https://zengold.org/...",
792,Zipper,ZIP,https://cryptorank.io/ico/zip,0.030,0.00,NaN,,NaT,NaT,,...,123210.0,Ethereum,Blockchain Infrastructure,2018-05-29,0.28,NaT,1,1,"[http://zipper.io/, https://twitter.com/Zipper...",


In [374]:
investor_table['Investor Count'] = investor_table['Funds and Investors'].apply(len)
#investor_table.to_csv("../data/ico_addinvestor.csv")